In [1]:
import pandas
import matplotlib.pyplot as plt
pandas.set_option('display.float_format', lambda x: '%.4f' % x)
import numpy as np
import epydemiology as epy

Error: PyMSSQL package not available. Some features may not be available.


## Load most recent 2020 data
https://www.ncsbe.gov/results-data/absentee-data

UPDATED: 9/23

- REMOVE: pending/pending cure

In [3]:
df = pandas.read_csv('../data/absentee_20201103.csv', encoding = "ISO-8859-1")

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.columns

Index(['county_desc', 'voter_reg_num', 'ncid', 'voter_last_name',
       'voter_first_name', 'voter_middle_name', 'race', 'ethnicity', 'gender',
       'age', 'application_num', 'voter_street_address', 'voter_city',
       'voter_state', 'voter_zip', 'ballot_mail_street_address',
       'ballot_mail_city', 'ballot_mail_state', 'ballot_mail_zip',
       'other_mail_addr1', 'other_mail_addr2', 'other_city_state_zip',
       'relative_request_name', 'relative_request_address',
       'relative_request_city', 'relative_request_state',
       'relative_request_zip', 'election_dt', 'voter_party_code',
       'precinct_desc', 'cong_dist_desc', 'nc_house_desc', 'nc_senate_desc',
       'ballot_req_delivery_type', 'ballot_req_type', 'ballot_request_party',
       'ballot_req_dt', 'ballot_send_dt', 'ballot_rtn_dt', 'ballot_rtn_status',
       'site_name', 'sdr', 'mail_veri_status'],
      dtype='object')

In [5]:
len(df)

177132

FOR ALL DATA:
- Remove one-stop data
- ballot_req_type == "MAIL"
- valid return date only 

In [6]:
# it's all mail
df.ballot_req_type.unique()

array(['MAIL'], dtype=object)

In [7]:
# it's all returned already
df = df[df['ballot_rtn_dt'].notnull()]
len(df)

177132

In [10]:
df.ballot_rtn_status.unique()

array(['ACCEPTED', 'WITNESS INFO INCOMPLETE', 'RETURNED UNDELIVERABLE',
       'SPOILED', 'PENDING CURE', 'ACCEPTED - CURED', 'PENDING',
       'CONFLICT', 'SIGNATURE DIFFERENT', 'DUPLICATE'], dtype=object)

#### Remove pending cure, pending

In [13]:
df_nopending = df[~df.ballot_rtn_status.isin(['PENDING', 'PENDING CURE'])]
df_nopending.ballot_rtn_status.unique()

array(['ACCEPTED', 'WITNESS INFO INCOMPLETE', 'RETURNED UNDELIVERABLE',
       'SPOILED', 'ACCEPTED - CURED', 'CONFLICT', 'SIGNATURE DIFFERENT',
       'DUPLICATE'], dtype=object)

## Ballot breakdown by race

In [14]:
df_nopending.race.value_counts()

WHITE                                  128952
BLACK or AFRICAN AMERICAN               28808
UNDESIGNATED                            10420
OTHER                                    3239
ASIAN                                    2768
TWO or MORE RACES                         782
INDIAN AMERICAN or ALASKA NATIVE          523
NATIVE HAWAIIAN or PACIFIC ISLANDER         1
Name: race, dtype: int64

In [15]:
df_nopending.race.value_counts(normalize=True)

WHITE                                 0.7348
BLACK or AFRICAN AMERICAN             0.1642
UNDESIGNATED                          0.0594
OTHER                                 0.0185
ASIAN                                 0.0158
TWO or MORE RACES                     0.0045
INDIAN AMERICAN or ALASKA NATIVE      0.0030
NATIVE HAWAIIAN or PACIFIC ISLANDER   0.0000
Name: race, dtype: float64

## Ballots rejected

In [16]:
df_nopending[df_nopending.race=="BLACK or AFRICAN AMERICAN"]['ballot_rtn_status'].value_counts()

ACCEPTED                   27975
WITNESS INFO INCOMPLETE      533
SPOILED                      229
ACCEPTED - CURED              50
RETURNED UNDELIVERABLE        17
SIGNATURE DIFFERENT            2
CONFLICT                       1
DUPLICATE                      1
Name: ballot_rtn_status, dtype: int64

In [17]:
1- df_nopending[df_nopending.race=="BLACK or AFRICAN AMERICAN"]['ballot_rtn_status'].value_counts(normalize=True)

ACCEPTED                  0.0289
WITNESS INFO INCOMPLETE   0.9815
SPOILED                   0.9921
ACCEPTED - CURED          0.9983
RETURNED UNDELIVERABLE    0.9994
SIGNATURE DIFFERENT       0.9999
CONFLICT                  1.0000
DUPLICATE                 1.0000
Name: ballot_rtn_status, dtype: float64

In [18]:
17537- 16662

875

In [19]:
df_nopending[df_nopending.race=="WHITE"]['ballot_rtn_status'].value_counts()

ACCEPTED                   127551
SPOILED                       796
WITNESS INFO INCOMPLETE       334
ACCEPTED - CURED              195
RETURNED UNDELIVERABLE         65
CONFLICT                        6
SIGNATURE DIFFERENT             3
DUPLICATE                       2
Name: ballot_rtn_status, dtype: int64

In [20]:
1-df_nopending[df_nopending.race=="WHITE"]['ballot_rtn_status'].value_counts(normalize=True)

ACCEPTED                  0.0109
SPOILED                   0.9938
WITNESS INFO INCOMPLETE   0.9974
ACCEPTED - CURED          0.9985
RETURNED UNDELIVERABLE    0.9995
CONFLICT                  1.0000
SIGNATURE DIFFERENT       1.0000
DUPLICATE                 1.0000
Name: ballot_rtn_status, dtype: float64